# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))
    
    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')


ModuleNotFoundError: No module named 'mplleaflet'

In [2]:
from matplotlib import pyplot, dates, ticker
import datetime
import pylab 
import numpy as np
%matplotlib inline

df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
res = df.groupby('Date')['Data_Value'].agg(['max','min']).reset_index()
res["Date"] = pd.to_datetime(res['Date'])

column_1 = res.ix[:,0]

DataFrame=pd.DataFrame({"year": column_1.dt.year,
              "month": column_1.dt.month,
              "day": column_1.dt.day,
              "hour": column_1.dt.hour,
              "dayofyear": column_1.dt.dayofyear,
              "week": column_1.dt.week,
              "weekofyear": column_1.dt.weekofyear,
              "dayofweek": column_1.dt.dayofweek,
              "weekday": column_1.dt.weekday,
              "quarter": column_1.dt.quarter,
             })

res['dayofyear']=DataFrame['dayofyear']
res['Month']=DataFrame['month']
res['Year']=DataFrame['year']
res2=res[res['Year']>2014].reset_index()
res=res[res['Year']!=2015]

resMax = res.groupby('dayofyear')['max'].max().reset_index()
resMin = res.groupby('dayofyear')['min'].min().reset_index()


req= pd.concat([res['dayofyear'], res['Month'], resMax['max'], resMin['min']], axis=1, keys=['Date', 'Month','MaxTemp', 'MinTemp'])

req=req[~np.isnan(req).any(axis=1)]
req=req[0:365]

res3=res2[res2['max']>req['MaxTemp']]
res4=res2[res2['min']<req['MinTemp']]

# axis tooling
locator = dates.MonthLocator()
formatter = dates.DateFormatter('%b')

# plot things, use the locators
fig, ax = pyplot.subplots(figsize=(20,10))

date=req['Date']
ax.plot_date(date, req['MaxTemp'], '-', label='Max Temp(2005-2014)')
ax.plot_date(date, req['MinTemp'], '-', label='Min Temp(2005-2014)')
ax.fill_between(req['Date'], req['MaxTemp'], req['MinTemp'], facecolor='#fdf8ae')

date3=res3['dayofyear']
date4=res4['dayofyear']
ax.plot_date(date3, res3['max'], 'o', markersize=10, label='Breaking Record High(2015)')
ax.plot_date(date4, res4['min'], 'o', markersize=10, label='Breaking Record Low(2015)')

ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)
pyplot.gcf().subplots_adjust(bottom=0.1)


ax.xaxis.set_ticks(np.arange(1, 365, 31))
ax.set_xlim(xmin=1,xmax=365)

# Customize minor tick labels
ax.set_xticks(np.arange(15.5, 365, 31),minor=True)
ax.set_xticklabels(['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec'], minor=True, size=14)

plt.tick_params(axis='y', which='major', labelsize=14)
pylab.ylabel("Temperature (℃)", size=16)



plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    ) # labelbottom='off' labels along the bottom edge are off
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='major',      # both major and minor ticks are affected
    bottom='on',      # ticks along the bottom edge are off
    labelbottom='off' ) #
xticks = ax.xaxis.get_major_ticks()
xticks[0].set_visible(False)


pylab.title("Range of Temperature Variations by Day of the Year over the Period 2005-2014 \n Breaking Points in 2015 are Shown with Dots",size=16)
ttl = ax.title
ttl.set_position([.5, 1.02])
pylab.xlabel("Month", size=16, labelpad=18)

pylab.legend(loc='lower center',prop={'size':16})
pylab.show()


NameError: name 'pd' is not defined